In [ ]:
%reset -f 
dataframefile = "../sample_data/dataframes_sample.pkl"
samplename = "twomg"
highbinding_glycans = {75,
  163,
  185,
  336,
  337,
  372,
  543,
  545,
  547,
  549,
  550,
  551,
  552,
  565,
  566,
  569,
  572,
  576,
  577,
  578,
  579,
  580,
  581,
  582,
  583,
  584,
  585,
  586,
  587,
  588,
  589}
highbinding_glycans = {543,
 547,
 549,
 550,
 551,
 565,
 566,
 569,
 577,
 578,
 579,
 580,
 581,
 582,
 588,
 589}

### Notebook arguments

- **dataframefile** (*string*): which pickled data file to load

- **samplename** (*string*): name of particular experiment

- **highbinding_glycans** (*list or set *): list of integers which refer to the id of glycans as found in the dataframe file

# MCAW analysis


Pass in glycan ids and will look up the glycan IUPAC from a dataframe, and then get the KCF's and pass these to the RINGS MCAW tool. 

You can either run this notebook directly, or run it through the [meta notebook](meta_run_mcaw_analysis.ipynb) for batch processing.


In [ ]:
## import all required dependencies

In [ ]:
# standard imports
import urllib
import urllib2
import os
import sys
import json
import StringIO
import pickle

# dataframe and numerical
import pandas as pd
import numpy as np

# plotting
import matplotlib.pyplot as plt
%matplotlib inline
# plotting params
from matplotlib import rcParams
rcParams["figure.figsize"] = 10, 8

#scipy
from scipy import stats
from scipy.special import erf
from scipy import sqrt

# glypy
import glypy
from glypy.plot import plot
from glypy.io import glycoct

# other modules
sys.path.append('../scripts/')
import post_glycan_convert as pgc
import post_kcf_to_image as pki
import post_mcaw as pm

# iPython
from IPython.display import Image,display, HTML

In [ ]:
# load dataframe
subdir="./"

if not os.path.isfile(dataframefile):
    exit(1)
with open(os.path.join(subdir, dataframefile)) as f:
    dataframes = pickle.load(f)
    
    
dataframes[0]["sample"]
frame=dataframes[0]["dataframe"]
frame.head()

In [ ]:
# create a data frame with the glycan and the spacer as separate columns

Structure="Structure on Masterlist"

df=pd.concat([frame["Chart Number"], frame[Structure]], axis=1)
              
df.head()
#frame["Structure"]
df[Structure].str.extract('(.*-)')
df["Glycan"]=df[Structure].str.extract('(.*-)').str.strip('-')

#df['Structure'].str.extract('(-Sp.+?$)')
df["Spacer"]=df[Structure].str.split('.*-').str[1]
df

In [ ]:
# create a function that plots using glypy

def plotter(func):
    """
    A decorator that plots the function .
    (it actually just prints it, but it could be logging!)
    """
    def wrapper(*args, **kwargs):
        res = func(*args, **kwargs)
        #print func.__name__, args, kwargs
        #plot(res,label=True)
        return res
    return wrapper

@plotter
def get_gly_iupac(iupacstring):
    try:
        kchandle = StringIO.StringIO(iupacstring)
        kcf=pgc.mechanise_glycan_convert(kchandle, "Kcf", "text")
        kchandle2 = StringIO.StringIO(kcf)
        gct=pgc.mechanise_glycan_convert(kchandle2, "Glycoct", "text")
        return glycoct.loads(gct)
    except:
        return iupacstring

In [ ]:
# create a dataframe for the highbinders
highbinders_df = df[df["Chart Number"].isin(highbinding_glycans)]


In [ ]:
# Not always able to get iupac-> kcf-> glycoct
# why not just pull imaged from kcf-> image

def get_gly_image(iupacstring):
    try:
        kchandle = StringIO.StringIO(iupacstring)
        kcf=pgc.mechanise_glycan_convert(kchandle, "Kcf", "text")
        kchandle2 = StringIO.StringIO(kcf)
        gct=pki.post_rings_kcf_to_image(kchandle2)
        return pki.get_first_image_from_html(gct).encode("base64")
    except:
        return iupacstring

aimg= get_gly_image(df["Glycan"][372])

Image(aimg) 

In [ ]:

highbinders_df["Image"]=highbinders_df["Glycan"]
# create a function called to http://chrisalbon.com/python/pandas_apply_operations_to_dataframes.html
# maps apply to the showl frame.... 
def map_image(x):
    # that, if x is a string,
    return '<html><img src="data:image/png;base64,'+urllib.quote(get_gly_image(x).rstrip('\n'))+'"/></html>'
    #return '<html><img src="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAcIAAACWCAYAAABNcIgQAAAHM0lEQVR4nO3d32vXhR7H8ZdaSwoxCSLCYEOaw12YtIIIphe7S/+C8iIIIQI3+6GjIJ0gbWL12SC6MAhd3Xhp367sotKKKLd1EWOSFARRQiCMaivmztWR0+Gc06mjfj2+H4/r7fN5fQbjyeez7bMVy8vLywGAola2ewAAtJMQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhACUJoQAlCaEAJQmhBCG83NzWVubq7dM6A0IYQ2OHPmTPr6+nL//ffnvvvuS19fXz766KN2z4KShBCuoQsXLmTXrl3ZunVrzp49m+7u7vT09OTs2bPp7+/Prl27cuHChXbPhFKEEK6BX375JQcOHEhXV1eOHj2azs7OnDx5MlNTU5mens7JkyfT2dmZo0ePpqurKwcOHMjCwkK7Z0MJQghX0fLyco4fP57u7u6MjIzkpptuStM0mZ2dzY4dOy5/3I4dOzI7O5umabJq1aqMjIzk3nvvzfHjx7O8vNzGK4Ab34pl32VwVczMzGRoaCgffPBBVq5cmSeeeCL79+/P3Xff/R8/77vvvsvIyEjeeOONXLp0KVu3bs34+Hg2b958jZZDLUIIV9gPP/yQ5557Lm+//XYuXbqUhx9+OE3TpK+v708d5/PPP8/g4GA+/vjjrFy5Mo8++miOHDmSO++88yoth5o8GoUrZHFxMWNjY+np6cnk5GTWr1+fEydO5PTp0386gknS19eXM2fO5MSJE1m/fn0mJyezcePGjI2NZXFx8SpcAdTkjhCugFarlWeeeSbnzp3L6tWrs2/fvuzduze33nrrFTn+zz//nMOHD2dsbCwLCwvp7u7Oyy+/nO3bt1+R40NlQgj/gy+//DJDQ0N57733kiQ7d+7MoUOHcs8991yV83377bd54YUXMjk5mSQZGBhI0zTp7e29KueDCoQQ/oKLFy9m//79ef311/Pbb79l8+bNaZom27Ztuybnf//99zM0NJQvvvgiN998c5588skcPHgwa9euvSbnhxuJnxHCn7C0tJTx8fFs2LAhExMTWbduXY4dO5apqalrFsEk2bZtW6ampnLs2LGsW7cuExMT2bBhQ8bHx7O0tHTNdsCNwB0h/Jc+/PDDDA4OZmZmJh0dHdmzZ0+Gh4dz++23t3XXxYsXMzo6mldffTW//vprtmzZkqZp0t/f39Zd8P9CCOEPfP3119m9e3darVaS5JFHHskrr7yS7u7uNi/7vXPnzuXpp5/Ou+++myTZvn17JiYm0tXV1eZlcH3zaBT+jfn5+QwPD6e3tzetViu9vb05depUWq3WdRfBJOnu7k6r1cqpU6eyadOmy5uHh4czPz/f7nlw3RJC+Cd/fy1aT09PxsbG0tHRkaZpMj09nYGBgXbP+0MDAwOZmZlJ0zTp6Oi4/LeNXtcG/5pHo/APPvnkkwwODuazzz7LqlWr8tRTT+XFF1/MHXfc0e5pf8mPP/6YgwcP5rXXXsvS0lIeeOCBjI+P56GHHmr3NLhuCCEk+f7777N379689dZbWV5eTn9/f5qmyZYtW9o97YqYnp7O4OBgTp8+nRUrVuSxxx7L4cOHc9ddd7V7GrSdEFLawsJCRkdHc+TIkfz000/p7OzMxMTE7/4zxI3knXfeye7du/PNN9/ktttuy7PPPpvh4eGsXr263dOgbYSQsj799NM8/vjjmZ2dzS233JI9e/bk+eefz5o1a9o97aqan5/PoUOH0jRNFhcXs2nTprz55pt58MEH2z0N2sIvy1DW2rVrc/78+ezcuTPnz5/PSy+9dMNHMEnWrFmT0dHRy9f+1VdfeSMNpbkjpLS5ubls3Lix3TPayteA6oQQgNI8GgWgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKA0IQSgNCEEoDQhBKC0vwFUhtrR8bi1XgAAAABJRU5ErkJggg=="/></html>'


newslice = highbinders_df['Image'].map(map_image) # http://chrisalbon.com/python/pandas_apply_operations_to_dataframes.html

In [ ]:
pd.set_option('display.max_colwidth', 10000)

newslice
HTML(newslice.to_frame().to_html(escape=False))

In [ ]:
def get_gly_kcf(iupacstring):
    try:
        kchandle = StringIO.StringIO(iupacstring)
        kcf=pgc.mechanise_glycan_convert(kchandle, "Kcf", "text")
        return kcf
    except:
        return iupacstring
    
highbinders_df["KCF"]=highbinders_df["Glycan"]
# create a function called to http://chrisalbon.com/python/pandas_apply_operations_to_dataframes.html
# maps apply to the showl frame.... 
def map_kcf(x):
    # that, if x is a string,
    return get_gly_kcf(x)
    


newslice2 = highbinders_df['KCF'].map(map_kcf) # http://chrisalbon.com/python/pandas_apply_operations_to_dataframes.html

In [ ]:
kcfdict=newslice2.to_dict()
kcfdict.keys()
kcflist=[]
for key in kcfdict:
    kcflist.append(kcfdict[key])


In [ ]:
#for i in kcflist:
kcflist2=[]
for i in kcflist:
     kcflist2.append(i.strip('\n'))

    
    

In [ ]:
print "\n".join(kcflist2) # can paste this into http://rings.t.soka.ac.jp/cgi-bin/tools/MCAW/mcaw_index.pl

In [ ]:
print "".join(kcflist)



In [ ]:

kcfhandle = StringIO.StringIO("".join(kcflist))
mcaw_html = pm.post_rings_mcaw(kcfhandle)
mcaw_img = pm.get_image_from_html(mcaw_html)
                              

#Image(mcaw_img) 


In [ ]:
#help(Image)
Image(mcaw_img,width=3000) # double click on image to resize

In [ ]:
        #return iupacstring

In [ ]:


# Also send all the kcfs and return pics (just in case table above breaks)
def get_gly_image_html(kcfstring):
    try:
        kchandle2 = StringIO.StringIO(kcfstring)
        gct=pki.post_rings_kcf_to_image(kchandle2)
        return gct
    except Exception as e:
        return e
        #return iupacstring

kcfall = get_gly_image_html("".join(kcflist))

In [ ]:
HTML(kcfall)